<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/LocalMinimaEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
from torch.autograd.functional import hessian

In [2]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()

,x_2dvec,y
0,"[2.8,0.4]",1
1,"[3.1,4.3]",1
2,"[0.1,-3.4]",1
3,"[-4.2,-3.3]",1
4,"[-0.5,0.2]",1


In [3]:
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(X_raw.shape[0])

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
10


Neural Structure:
$M((x_0, x_1)) = \sigma(v_0σ(w_{0,0}x_0 + w_{0,1}x_1 + b_0) + v_1σ(w_{1,0}x_0 + w_{1,1}x_1 + b_1) + c) $

In [4]:
Image(url='https://github.com/zw2788/LocalMinimaConstruction/blob/main/221Sigmoid.png?raw=true')


In [5]:
def sigmoid(x):
    """Calculates sigmoid function."""
    return 1. / (1 + np.exp(-x))

# parameters between input layer and the hidden layer
W_0 = np.random.normal(size=(2, X_raw.shape[1]))
print(f"Shape of W_0 is {W_0.shape}")
print(W_0)
print(W_0.shape[1])
b = np.random.normal(size=(2, 1))
print(f"Shape of b_1 is {b.shape}")

# parameters between hidden layer and output layer
V_0 = np.random.normal(size=(1, W_0.shape[1]))
c = np.random.normal(size=(1, 1))

# calculate the forward propagation
Z_1 = X_raw @ W_0.T
print(f"\nShape of Z_1 is {Z_1.shape}")
print("Samples for Z_1:")
print(Z_1[:5])

A_1 = sigmoid(Z_1 + b.T)
print(f"Shape of A_1 is {A_1.shape}")
print("Samples for A_1:")
print(A_1[:5])

Z_2 = A_1 @ V_0.T
print(f"\nShape of Z_2 is {Z_2.shape}")
print("Samples for Z_2:")
print(Z_2[:5])

A_2 = Y_hat = sigmoid(Z_2 + c.T)
print(f"Shape of A_2 is {A_2.shape}")
print("Samples for A_2:")
print(A_2[:5])



Shape of W_0 is (2, 2)
[[-0.65999928 -0.83014452]
 [ 1.18497509 -0.77079179]]
2
Shape of b_1 is (2, 1)

Shape of Z_1 is (10, 2)
Samples for Z_1:
[[-2.18005581  3.00961355]
 [-5.61561923  0.3590181 ]
 [ 2.75649145  2.73918959]
 [ 5.51147392 -2.43328249]
 [ 0.16397074 -0.7466459 ]]
Shape of A_1 is (10, 2)
Samples for A_1:
[[0.08351261 0.95231842]
 [0.00292622 0.58510329]
 [0.92696632 0.93842404]
 [0.99501329 0.07954525]
 [0.48712181 0.31823277]]

Shape of Z_2 is (10, 1)
Samples for Z_2:
[[-0.28210437]
 [-0.17867608]
 [-0.18456352]
 [ 0.08571825]
 [-0.04347836]]
Shape of A_2 is (10, 1)
Samples for A_2:
[[0.1821991 ]
 [0.19811929]
 [0.19718562]
 [0.24347962]
 [0.22047631]]


In [12]:
def forward_prop(
    X_raw: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Performs the forward propagation of the given NN."""
    # Note the NN structure is passed in from outside.
    Z_1 = X_raw @ W_0.T
    A_1 = sigmoid(Z_1 + b.T)

    Z_2 = A_1 @ V_0.T
    A_2 = sigmoid(Z_2 + c.T)

    return A_2, Z_2, A_1, Z_1

#Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)

In [7]:
def derivatives_by_backprop(
    X_raw: np.array,
    Y: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Calculates the derivatives of the parameters by backforward propagation.

    Here we assume it is a binary classification problem, with sigmoid activation functions.
    """
    # forward propagation
    dW_0, db, dV_0, dc = 0, 0, 0, 0
    Y_hat, Z_2, A_1, Z_1 = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
    n = len(Y_hat)

    loss = -np.mean(np.multiply(Y, np.log(Y_hat)) + np.multiply(1 - Y, np.log(1 - Y_hat)))

    dZ_2 = Y_hat - Y
    dV_0 = dZ_2.T @ A_1 / n
    dc = np.mean(dZ_2.T, axis=1, keepdims=True)

    dZ_1 = np.multiply(dZ_2 @ V_0, np.multiply(A_1, 1 - A_1))
    dW_0 = (dZ_1.T @ X_raw) / n
    db = np.mean(dZ_1.T, axis=1, keepdims=True)

    return dW_0, db, dV_0, dc, loss

dW_0, db, dV_0, dc, loss = derivatives_by_backprop(X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c)

In [8]:
def gradient_descent(
    X_raw: np.array,
    Y: np.array,
    W_0_init: np.array,
    b_init: np.array,
    V_0_init: np.array,
    c_init: np.array,
    learning_rate: float = 0.01,
    epsilon: float = 1e-6,
    verbose: bool = False,
) -> Tuple:
    """Runs gradient descent to fit the NN via backprop."""
    W_0 = W_0_init
    b = b_init
    V_0 = V_0_init
    c = c_init
    losses = [float("inf"), ]
    roc_auc_scores = [0.5, ]

    diff_in_loss = float("inf")
    iteration = 0
    while abs(diff_in_loss) > epsilon:
        iteration += 1
        dW_0, db, dV_0, dc, loss = derivatives_by_backprop(
            X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c
        )

        W_0 -= learning_rate * dW_0
        b  -= learning_rate * db
        V_0 -= learning_rate * dV_0
        c  -= learning_rate * dc

        losses.append(loss)
        diff_in_loss = losses[-1] - losses[-2]

        Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
        roc_auc = roc_auc_score(y_true=Y, y_score=Y_hat)
        roc_auc_scores.append(roc_auc)

        if verbose and iteration % 10 == 0:
            print(loss, roc_auc)
    return W_0, b, V_0, c, losses

In [9]:
# parameters for the first layer
W_0_init = np.array([[1.06*(1+0.001*np.random.choice([-1,1])),-0.037*(1+0.001*np.random.choice([-1,1]))],[-0.056*(1+0.001*np.random.choice([-1,1])),1.095*(1+0.001*np.random.choice([-1,1]))]])
b_init = np.array([[-0.051*(1+0.001*np.random.choice([-1,1]))],[-0.0689*(1+0.001*np.random.choice([-1,1]))]])

# parameters for the second layer

V_0_init = np.array([[3.769*(1+0.001*np.random.choice([-1,1])),-3.72*(1+0.001*np.random.choice([-1,1]))]])
c_init = np.array([[-0.0148*(1+0.001*np.random.choice([-1,1]))]])

#random initial points
#np.random.seed(42)
#W_0_init = np.random.normal(size=(2, X_raw.shape[1]))
#b_init  = np.random.normal(size=(2, 1))
#V_0_init = np.random.normal(size=(1, W_0.shape[1]))
#c_init  = np.random.normal(size=(1, 1))

#initial points from all zero
#W_0_init = np.array([[0.1,0.1],[0.1,0.1]])
#b_init = np.array([[0.1],[0.1]])
#V_0_init = np.array([[0.1,0.1]])
#c_init = np.array([[0.1]])

W_0, b, V_0, c, losses = gradient_descent(
    X_raw=X_raw,
    Y=Y,
    W_0_init=W_0_init,
    b_init =b_init,
    V_0_init=V_0_init,
    c_init =c_init,
    learning_rate=0.05,
    epsilon=1e-8,
    verbose=True,
)
print(W_0.T)
print(b)
print(V_0.T)
print(c)
print(losses)

[[ 1.05894267 -0.05605827]
 [-0.03707451  1.09391677]]
[[-0.0510536 ]
 [-0.06882748]]
[[ 3.76522613]
 [-3.71629306]]
[[-0.01479446]]
[inf, 0.5777382528970282, 0.5777382434998941]


Remarks:
1.Not a global minimum. (details see deepmind paper's proof.)
2.

What a few first failed attempts made us realize was, that the nature of a successful example would
have to be both geometric and analytic at the same time. What “deadlocks” a sigmoid-based neural
network is not only the geometric configuration of the points, but also the very precise cross-ratios of
distances between them. The successful construction of the presented example was a combination of
studying the failed attempts generated by a guesswork and then trying to block the “escape routes”
with a gradient descent in the data space. Once a “close enough” configuration of points was deduced, the gradient descent was applied to the datapoints (in the data space) in order to minimize the
length of the gradient in the weights space of the loss function. This procedure modifies the dataset
in such a way that the (fixed) set of weights becomes a critical point of the error surface, but starting
from a randomly chosen dataset almost surely produces a saddle point, instead of a minimum. Using
the “close enough” configuration yields a higher chance of finding a true local minimum.


In [21]:
import torch

# Define the sigmoid function using PyTorch
def sigmoid_torch(x):
    return 1 / (1 + torch.exp(-x))

# Convert your forward propagation to work with PyTorch tensors
def forward_prop_torch(X_raw, W_0, b, V_0, c):
    Z_1 = torch.mm(X_raw, W_0.t())  # Use torch.mm for matrix multiplication
    A_1 = sigmoid_torch(Z_1 + b.t())

    Z_2 = torch.mm(A_1, V_0.t())
    A_2 = sigmoid_torch(Z_2 + c.t())

    return A_2, Z_2, A_1, Z_1

# Assume your X_raw, W_0, b, V_0, and c are given as numpy arrays
# Convert them to PyTorch tensors with requires_grad=True
X_raw_torch = torch.tensor(X_raw, dtype=torch.float32, requires_grad=True)
W_0_torch = torch.tensor(W_0, dtype=torch.float32, requires_grad=True)
b_torch = torch.tensor(b, dtype=torch.float32, requires_grad=True)
V_0_torch = torch.tensor(V_0, dtype=torch.float32, requires_grad=True)
c_torch = torch.tensor(c, dtype=torch.float32, requires_grad=True)

# Compute the forward propagation using the PyTorch version of the function
Y_hat, _, _, _ = forward_prop_torch(X_raw_torch, W_0_torch, b_torch, V_0_torch, c_torch)

# Define your loss function in PyTorch
Y_torch = torch.tensor(Y, dtype=torch.float32)  # Convert Y to a PyTorch tensor
loss = -torch.mean(Y_torch * torch.log(Y_hat) + (1 - Y_torch) * torch.log(1 - Y_hat))

# Now you can compute the gradients and Hessian as shown in the previous example

In [27]:
# ... previous code to compute loss ...

# Compute the gradient of the loss with respect to each parameter
grad_W_0 = torch.autograd.grad(loss, W_0_torch, create_graph=True)[0]
grad_b = torch.autograd.grad(loss, b_torch, create_graph=True)[0]
grad_V_0 = torch.autograd.grad(loss, V_0_torch, create_graph=True)[0]
grad_c = torch.autograd.grad(loss, c_torch, create_graph=True)[0]

# ... previous code to compute gradients ...

# Compute the Hessian for W_0
Hessian_W_0 = []
for grad in grad_W_0.view(-1):
    grad2_W_0_i = torch.autograd.grad(grad, W_0_torch, create_graph=True)[0].view(-1)
    Hessian_W_0.append(grad2_W_0_i)
Hessian_W_0 = torch.stack(Hessian_W_0).reshape(W_0_torch.numel(), W_0_torch.numel())

# Compute the Hessian for b
Hessian_b = []
for grad in grad_b.view(-1):
    grad2_b_i = torch.autograd.grad(grad, b_torch, create_graph=True)[0].view(-1)
    Hessian_b.append(grad2_b_i)
Hessian_b = torch.stack(Hessian_b).reshape(b_torch.numel(), b_torch.numel())

# ... similar blocks for V_0 and c ...

# Calculate eigenvalues for each Hessian matrix
eigenvalues_W_0 = torch.linalg.eigvals(Hessian_W_0)
eigenvalues_b = torch.linalg.eigvals(Hessian_b)
# ... similar lines for V_0 and c ...

# Since the eigenvalues may be complex, you might want to get their real parts
eigenvalues_W_0 = eigenvalues_W_0.real
eigenvalues_b = eigenvalues_b.real
# ... similar lines for V_0 and c ...

# Print out the eigenvalues
print("Eigenvalues for W_0:", eigenvalues_W_0)
print("Eigenvalues for b:", eigenvalues_b)
# ... similar print statements for V_0 and c ...
# ... previous code for W_0 and b ...

# Compute the Hessian for V_0
Hessian_V_0 = []
for grad in grad_V_0.view(-1):
    grad2_V_0_i = torch.autograd.grad(grad, V_0_torch, create_graph=True)[0].view(-1)
    Hessian_V_0.append(grad2_V_0_i)
Hessian_V_0 = torch.stack(Hessian_V_0).reshape(V_0_torch.numel(), V_0_torch.numel())

# Compute the Hessian for c
Hessian_c = []
for grad in grad_c.view(-1):
    grad2_c_i = torch.autograd.grad(grad, c_torch, create_graph=True)[0].view(-1)
    Hessian_c.append(grad2_c_i)
Hessian_c = torch.stack(Hessian_c).reshape(c_torch.numel(), c_torch.numel())

# Calculate eigenvalues for each Hessian matrix
eigenvalues_V_0 = torch.linalg.eigvals(Hessian_V_0)
eigenvalues_c = torch.linalg.eigvals(Hessian_c)

# Since the eigenvalues may be complex, you might want to get their real parts
eigenvalues_V_0 = eigenvalues_V_0.real
eigenvalues_c = eigenvalues_c.real

# Print out the eigenvalues
print("Eigenvalues for V_0:", eigenvalues_V_0)
print("Eigenvalues for c:", eigenvalues_c)

Eigenvalues for W_0: tensor([0.5636, 0.0751, 0.0194, 0.4179], grad_fn=<SelectBackward0>)
Eigenvalues for b: tensor([0.0090, 0.0991], grad_fn=<SelectBackward0>)
Eigenvalues for V_0: tensor([0.1466, 0.0059], grad_fn=<SelectBackward0>)
Eigenvalues for c: tensor([0.1976], grad_fn=<SelectBackward0>)
